In [1]:
import gymnasium as gym
from gymnasium.envs.registration import register
import mujoco
from tqdm import tqdm
import torch
import math

from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList


In [2]:
register(
    id="galaxea_r1Pro",
    entry_point="galaxea_r1Pro:Galaxea_r1Pro",
)


In [ ]:
# 仿真交互环境
train_env = gym.make("galaxea_r1Pro")
obs, _ = train_env.reset()
print(f"obs space: {train_env.observation_space.shape}, action space: {train_env.action_space.shape}")


log_dir = "./tb_log/"

total_timesteps = 2400000  # 总训练步数

# 评估环境
eval_env = gym.make("galaxea_r1Pro")   # 你的环境

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=log_dir+"best_model2",  # 自动保存最优模型的目录
    log_path=log_dir,                        # 保存评估日志
    eval_freq=total_timesteps // 10,                          # 每 1 万步评估一次
    n_eval_episodes=5,                         # 每次评估 5 个 episode
    deterministic=True,                        # 评估时用确定性策略
    render=False
)


In [ ]:
class RewardInfoCallback(BaseCallback):
    """
    在 TensorBoard 中记录 info 字典中的各个 reward 项。
    支持 vectorized environments。
    """
    def __init__(self, verbose=0):
        super().__init__(verbose)

    def _on_step(self) -> bool:
        infos = self.locals.get("infos", [])
        if not infos:
            return True

        # 对 vectorized env 平均 reward
        reward_sums = {}
        for info in infos:
            for key, value in info.items():
                if key.startswith("reward_"):
                    reward_sums[key] = reward_sums.get(key, 0.0) + float(value)

        for key, total in reward_sums.items():
            mean_value = total / len(infos)
            self.logger.record(f"reward/{key}", mean_value)

        return True
    
reward_info_cb = RewardInfoCallback()
    

In [ ]:

# 自定义SAC网络结构
# obs space: (706,), action space: (24,)
policy_kwargs = dict(
    net_arch=dict(pi=[256, 256], qf=[256, 256]), # 每个隐藏层的神经元数量，也可以写成 [400, 300] 等
    activation_fn=torch.nn.ReLU  # 激活函数，可改为 torch.nn.Tanh
)

def warm_sin_lr(progress_remaining: float) -> float:
    """
    progress_remaining: 1 -> 0
    假设总共训练T步：
      - 前10% steps: 线性从 1e-5 升到 3e-4 (warm-up)
      - 之后: 按正弦方式从 3e-4 降到 1e-5
    """
    lr_min = 1e-4   
    lr_max = 1e-3
    warm_ratio = 0.01  # 10% warm-up

    # progress_remaining=1 -> step=0; progress_remaining=0 -> step=end
    progress_done = 1.0 - progress_remaining

    if progress_done < warm_ratio:
        # warm-up: 线性上升
        return lr_min + (lr_max - lr_min) * (progress_done / warm_ratio)
    else:
        # sin下降：这里重新归一化到[0,1]
        x = (progress_done - warm_ratio) / (1 - warm_ratio)
        return lr_min + (lr_max - lr_min) * math.sin((1 - x) * math.pi / 2)

model = SAC(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=warm_sin_lr,
    buffer_size=2_000_000,      # 经验回放缓冲区大小. 这个参数PPO没有
    learning_starts=1000, 
    batch_size=256,             # 默认256
    tau=0.005,                  # 软更新系数
    gamma=0.99,                 # 折扣因子
    train_freq=1,               # 每步都训练，采集多少个环境步的数据后训练一次
    gradient_steps=1,           # 对replayBuffer中读取到的batch，进行多少次梯度下降更新
    tensorboard_log=log_dir,   # 日志目录
    policy_kwargs=policy_kwargs,  # 将自定义结构传进去
)


# 训练模型, total_timesteps自行调整
model.learn(total_timesteps=total_timesteps, 
            tb_log_name="sac", 
            progress_bar=True,
            callback=[eval_callback, reward_info_cb])
# 保存模型
model.save("galaxea_sac_lr_forward")
model.save_replay_buffer("my_buffer.pkl")

In [ ]:
# resume training
model = SAC.load("galaxea_sac_lr_forward", env=train_env, tensorboard_log=log_dir)
model.load_replay_buffer("my_buffer.pkl")
model.learning_starts = 0  # 继续训练时，不需要再等待采集数据了,因为已经用了之前的replaybuffer
model.learning_rate = 1e-4
model.learn(total_timesteps=240000,
            tb_log_name="sac",
            progress_bar=True,
            callback=[eval_callback, reward_info_cb])


### 测试模型可视化

In [ ]:
# 使用可视化界面记录显示SAC测试结果
# 加载模型
model = SAC.load("./tb_log/best_model/best_model.zip")
# 创建测试环境
visual_env = gym.make("galaxea_r1Pro", render_mode="human")

for i in range(5):
    # 测试模型
    obs, info = visual_env.reset()
    cum_reward = 0
    for _ in tqdm(range(1500)):
        visual_env.render()
        action, _ = model.predict(obs, deterministic=True)
        next_obs, reward, terminated, truncated, info = visual_env.step(action)
        cum_reward += reward
        if terminated or truncated:
            print("累积奖励: ", cum_reward)
            break
            
        obs = next_obs

visual_env.close()

### 环境debug

In [ ]:
# 可视化环境
train_env = gym.make("galaxea_r1Pro", render_mode="human")
unwrapped_env = train_env.unwrapped

model = unwrapped_env.model  # MjModel

print(f"obs space: {train_env.observation_space.shape}, action space: {train_env.action_space.shape}")
print(f"action range: {train_env.action_space.low} to {train_env.action_space.high}")


print(f"actuator size: {model.nu}, ctrl_size: {unwrapped_env.data.ctrl.shape}")  # actuators and muscles
print(f"obs space: {train_env.observation_space.shape}, action space: {train_env.action_space.shape}")


# 随机采样动作
obs, _ = train_env.reset()
for _ in tqdm(range(1000)):
    train_env.render()
    action = train_env.action_space.sample()
    obs, reward, terminated, truncated, info = train_env.step(action)
    if terminated or truncated:
        obs, _ = train_env.reset()
train_env.close()


In [3]:
train_env = gym.make("galaxea_r1Pro")
obs, _ = train_env.reset()
unwrapped_env = train_env.unwrapped

mj_model = unwrapped_env.model  # MjModel

print(f"qpos size: {mj_model.nq}, qvel size: {mj_model.nv}, num_joints: {mj_model.njnt}")  # 都是旋转关节，所以这一项都相同
print(f"actuator size: {mj_model.nu}, ctrl_size: {unwrapped_env.data.ctrl.shape}")  # actuators and muscle
print(f"body_size: {mj_model.nbody}, body pos size: {unwrapped_env.data.xipos.shape}")  # nbody, 3

# print(f"action range: {env.action_space.low} to {env.action_space.high}")


qpos_idx = 0
for joint_id in range(mj_model.njnt):
    joint_name = mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_JOINT, joint_id)
    joint_type = mj_model.jnt_type[joint_id]
    
    # 根据关节类型确定占用的 qpos 数量
    if joint_type == mujoco.mjtJoint.mjJNT_FREE:    # 自由关节：7个qpos (x,y,z,qw,qx,qy,qz)
        for i, coord in enumerate(['x', 'y', 'z', 'qw', 'qx', 'qy', 'qz']):
            print(f"qpos[{qpos_idx:2d}]: {joint_name}_{coord}")
            qpos_idx += 1
    elif joint_type == mujoco.mjtJoint.mjJNT_HINGE:  # 铰链关节：1个qpos
        print(f"qpos[{qpos_idx:2d}]: {joint_name}")
        qpos_idx += 1
    elif joint_type == mujoco.mjtJoint.mjJNT_SLIDE:  # 滑动关节：1个qpos  
        print(f"qpos[{qpos_idx:2d}]: {joint_name}")
        qpos_idx += 1


data  = mujoco.MjData(mj_model)
mujoco.mj_forward(mj_model, data)  # 必须有这一步
pos = data.xipos            # shape = (nbody, 3)
x, y, z = pos[:, 0], pos[:, 1], pos[:, 2]
names = [mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_BODY, i)
         for i in range(mj_model.nbody)]

for i, name in enumerate(names):
    print(f"body[{i:2d}]: {name}, pos=({x[i]:.3f}, {y[i]:.3f}, {z[i]:.3f})")


train_env.close()

qpos size: 27, qvel size: 27, num_joints: 27
actuator size: 21, ctrl_size: (21,)
body_size: 26, body pos size: (26, 3)
qpos[ 0]: base_x
qpos[ 1]: base_y
qpos[ 2]: base_yaw
qpos[ 3]: steer_motor_joint1
qpos[ 4]: wheel_motor_joint1
qpos[ 5]: steer_motor_joint2
qpos[ 6]: wheel_motor_joint2
qpos[ 7]: steer_motor_joint3
qpos[ 8]: wheel_motor_joint3
qpos[ 9]: torso_joint1
qpos[10]: torso_joint2
qpos[11]: torso_joint3
qpos[12]: torso_joint4
qpos[13]: left_arm_joint1
qpos[14]: left_arm_joint2
qpos[15]: left_arm_joint3
qpos[16]: left_arm_joint4
qpos[17]: left_arm_joint5
qpos[18]: left_arm_joint6
qpos[19]: left_arm_joint7
qpos[20]: right_arm_joint1
qpos[21]: right_arm_joint2
qpos[22]: right_arm_joint3
qpos[23]: right_arm_joint4
qpos[24]: right_arm_joint5
qpos[25]: right_arm_joint6
qpos[26]: right_arm_joint7
body[ 0]: world, pos=(0.000, 0.000, 0.000)
body[ 1]: base_link, pos=(-0.056, -0.000, 0.206)
body[ 2]: steer_motor_link1, pos=(0.169, 0.275, 0.108)
body[ 3]: wheel_motor_link1, pos=(0.169, 0.2